In [ ]:
import os
from pathlib import Path
import requests
from datetime import datetime

import kfp

from kfp.v2.dsl import pipeline
from kfp.v2 import compiler

import google.cloud.aiplatform as aiplatform


In [ ]:
# Cache pipeline results to avoid compute the same components for different runs
ENABLE_CACHING = False

PIPELINE_NAME = "check-kfp"
# Your Kubeflow's detail
TEMPLATE_PATH = "check_kfp_components.json"
# GCS Bucket to store artefacts
PIPELINE_ROOT = f"gs://check-fkp-{PROJECT_ID}"

# NB: these parameters must be added for your pipeline's args
PIPELINE_PARAMS = {
    "in_str": "temp_string",
    "in_int": 123,
    "in_float": 0.12345,
}

In [ ]:
#Load the component

component1_op = kfp.components.load_component_from_file("component1/component.yaml")
component2_op = kfp.components.load_component_from_file("component2/component.yaml")


# Define a pipeline and create a task from a component:
@pipeline(name=PIPELINE_NAME, pipeline_root=PIPELINE_ROOT)
def tmp_pipe(
    in_str: str,
    in_int: int,
    in_float: float,
):
    comp1 = component1_op(
        in_str=in_str,
        in_int=in_int,
        in_float=in_float,
    ).set_display_name("Component1")
    
    
    comp2 = component2_op(
        in_artifact_path=comp1.outputs["out_artifact_path"],
        in_dataset_path=comp1.outputs["out_dataset_path"],
        in_metrics_path=comp1.outputs["out_metrics_path"],
        in_model_path=comp1.outputs["out_model_path"],
        # out_metrics=comp1.outputs["out_metrics"],
        # out_artifact=comp1.outputs["out_artifact"],
    ).set_display_name("Component2")



In [ ]:
compiler.Compiler().compile(
    pipeline_func=tmp_pipe, package_path=TEMPLATE_PATH
)

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=PIPELINE_ROOT)

In [ ]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
JOBID = f"training-pipeline-{TIMESTAMP}"

# Pipeline job
pipeline_ = aiplatform.pipeline_jobs.PipelineJob(
    enable_caching=ENABLE_CACHING,
    display_name=PIPELINE_NAME,
    template_path=TEMPLATE_PATH,
    job_id=JOBID,
    parameter_values=PIPELINE_PARAMS,
)
# Submit
pipeline_.submit(service_account=SERVICE_ACCOUNT)